# Dilithium - Glitches

In [1]:
import logging
logging.basicConfig(level=logging.NOTSET)
logging.getLogger('io.github.alex1s.python-dilithium').setLevel(logging.WARNING)

we append the path so that we can use the chipwhisperer library

In [2]:
import sys
if '../../../software' not in sys.path:
    sys.path.append('../../../software')
sys.path

['/home/alexis/chipwhisperer.dilithium/jupyter/courses/fault101',
 '/usr/lib/python38.zip',
 '/usr/lib/python3.8',
 '/usr/lib/python3.8/lib-dynload',
 '',
 '/home/alexis/.venvs/chipwhisperer.dilithium/lib/python3.8/site-packages',
 '../../../software']

In [3]:
import chipwhisperer as cw
import importlib
dilithium = importlib.import_module("python-dilithium.dilithium")
dilithium.generic = importlib.import_module("python-dilithium.dilithium.generic")
logging.basicConfig(level=logging.NOTSET)
cw

<module 'chipwhisperer' from '/home/alexis/chipwhisperer.dilithium/jupyter/courses/fault101/../../../software/chipwhisperer/__init__.py'>

In [4]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_STM32F4'

NIST_SECURITY_LEVEL = 2

In [5]:
SS_VER="SS_VER_2_1"

In [6]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ../../../hardware/victims/firmware/simpleserial-dilithium-ref
make PLATFORM=$1 CRYPTO_TARGET=NONE CRYPTO_OPTIONS=NONE SS_VER=$2

DEBUG:asyncio:Using selector: EpollSelector


SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
make[1]: '.dep' is up to date.
SS_VER set to SS_VER_2_1
.
Welcome to another exciting ChipWhisperer target build!!
arm-none-eabi-gcc (15:9-2019-q4-0ubuntu1) 9.2.1 20191025 (release) [ARM/arm-9-branch revision 277599]
Copyright (C) 2019 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling:
-en     simpleserial-dilithium-ref.c ...
mkdir -p "objdir-CW308_STM32F4"
-e Done!
.
Compiling:
-en     .././simpleserial/simpleserial.c ...
mkdir -p "objdir-CW308_STM32F4"
-e Done!
.
Compiling:
-en     .././hal/stm32f4/stm32f4_hal.c ...
mkdir -p "objdir-CW308_STM32F4"


In file included from .././hal/stm32f4/stm32f4_hal.c:3:
.././hal/stm32f4/stm32f4_hal_lowlevel.h:108: warning: "STM32F415xx" redefined
  108 | #define STM32F415xx
      | 
<command-line>: note: this is the location of the previous definition


-e Done!
.
Compiling:
-en     .././hal/stm32f4/stm32f4_hal_lowlevel.c ...
mkdir -p "objdir-CW308_STM32F4"


In file included from .././hal/stm32f4/stm32f4_hal_lowlevel.c:39:
.././hal/stm32f4/stm32f4_hal_lowlevel.h:108: warning: "STM32F415xx" redefined
  108 | #define STM32F415xx
      | 
<command-line>: note: this is the location of the previous definition


-e Done!
.
Compiling:
-en     .././hal/stm32f4/stm32f4_sysmem.c ...
mkdir -p "objdir-CW308_STM32F4"
-e Done!
.
Compiling:
-en     .././hal/stm32f4/stm32f4xx_hal_rng.c ...
mkdir -p "objdir-CW308_STM32F4"
-e Done!
.
Assembling: .././hal/stm32f4/stm32f4_startup.S
arm-none-eabi-gcc -c -mcpu=cortex-m4 -I. -x assembler-with-cpp -mthumb -mfloat-abi=soft -fmessage-length=0 -ffunction-sections -DF_CPU=7372800 -Wa,-gstabs,-adhlns=objdir-CW308_STM32F4/stm32f4_startup.lst -I.././simpleserial/ -I.././hal -I.././hal/stm32f4 -I.././hal/stm32f4/CMSIS -I.././hal/stm32f4/CMSIS/core -I.././hal/stm32f4/CMSIS/device -I.././hal/stm32f4/Legacy -I.././crypto/ .././hal/stm32f4/stm32f4_startup.S -o objdir-CW308_STM32F4/stm32f4_startup.o
.
LINKING:
-en     simpleserial-dilithium-ref-CW308_STM32F4.elf ...
-e Done!
.
Creating load file for Flash: simpleserial-dilithium-ref-CW308_STM32F4.hex
arm-none-eabi-objcopy -O ihex -R .eeprom -R .fuse -R .lock -R .signature simpleserial-dilithium-ref-CW308_STM32F4.elf simples

In [7]:
%run "../../Setup_Scripts/Setup_Generic_Dilithium.ipynb"

(ChipWhisperer NAEUSB WARNING|File naeusb.py:713) Your firmware (0.30) is outdated - latest is 0.64 See https://chipwhisperer.readthedocs.io/en/latest/firmware.html for more information
DEBUG:SimpleSerial2Dilithium:SimpleSerial2Dilithium logger says hello!


INFO: Found ChipWhisperer😍


In [8]:
fw_path = "../../../hardware/victims/firmware/simpleserial-dilithium-ref/simpleserial-dilithium-ref-{}.hex".format(PLATFORM)

In [9]:
#cw.program_target(scope, prog, fw_path)

In [10]:
# found using
# target.filter_msgs_one_iter([struct.pack('!H', i) for i in range(10)])
msgs_no_retry = [b'\x00\x01', b'\x00\x02', b'\x00\x07', b'\x00\t']
the_msg = msgs_no_retry[0]

In [11]:
def dilithium2_polyz_unpack_settings():
    scope.default_setup()
    scope.cglitch_setup() # default_setup for clock glitching
    scope.adc.samples = 24400 # the maximum for CWLite

def get_offset_first_iter():
    dilithium2_polyz_unpack_settings()
    target.reboot_flush()
    scope.arm()
    target.sign(msgs_no_retry[0])
    timed_out = scope.capture()
    assert not timed_out
    trig_count = scope.adc.trig_count
    # we have four coeffs (lvl2) per iteration and 256 coeffs in total, resulting in 64 iterations
    print(f'time per iteration: {trig_count / 64}')
    return round(trig_count / 64)


def get_correct_sig() -> bytes:
    """returns something around 500"""
    target.sign(msgs_no_retry[0])
    sig = target.get_sig()
    return sig

dilithium2_polyz_unpack_settings()
the_offset = get_offset_first_iter()
print(the_offset)
correct_sig = get_correct_sig()
_, z_correct, _ = dilithium.generic._unpack_sig(correct_sig, nist_security_level=2)

DEBUG:root:Trigger found at 77


time per iteration: 998.4375
998


## glitch log

Glitched NO 1!
	Ext offset: 498
	Offset: 7.03125
	Width: 5.078125
    
    
more to come ...

In [12]:
glitch_sig_no_1 = b'\xf3k\xa5.\xeb\xdb\xf2\xc1v\x94?6\x95\xfc\x1d\xefk\xc6p3 \xda+7\xa4\x1a\xe5"k\xbb\xbf\x17\xec\x87\x97\x07d\xa7\x1d\x85G\xb8tI\x8f\x01\x0b\xa3\xc3\xad\x02\x91\x82m\xab$\xe5\xba}\xa0<\xe0F\x95tA\x81|\xa6\xeeD\xfcl\xa7o>\n\x02\xf7u\x08p\x03_\xaa\xa80\xb5\xb8\x83H\xbcM\xf3\x14\ne\xa7\x96\x1d\xb5\xc3\xc3hJ\x13\x91i\x9a\xb00\x9fr>r\xda\xb8\xcd\xc5\x194d\xac\x82\x83):\x8a?"\xa3\xe2\x81C\xae\xa7K\x82K\xa3\xdaWUF0z\xec0\x9a\xd1\xa7$\xfb\x1a\x96\xa1~&F\x97\xcd~0\x83\xd9a<u\x9f\xd9\xa1{\x92H\xd1X\xe08\x96\xecT\x95\xbb\x1d\xc3\x98\xfct\xc6\xcc u\x0f\xcbN\x03\x05\xe9i^0\xb7\xf5\xbao\x1f\xfc\xc8\x12\xaa\xf7\xe8\x94\x1d\xecS\xa2A\xebp\xe8\xd0\x1c\xe0L\xdb\x9d\xf0\x18\x9b\xca\x8ah\x17\x9a^J\\\x90\x11\xa0\xf3:\xb1_3H\xed\x0b\xf2\x0eQ\x9a\x0c\xb87\xb6\xd4f\xbd\x7fj\xf4\'\x875\xe87\xf1\x8e\xaa\x05\xd0\xd7\x8f+&uAb\xbdS\xbb\n\xbc\xf6\xf7\xc2-\x03\xaf\x1c\x93\x89[+\xb0|O\xdf\xee&\x1a\xdcB@\xdf\xa3\xf7Qq\xe3\x0b|^dKdC\xa3\xf4\xbc\xe5\x19\x85\x9c\x8a\xbd\x99,\xcf:\xb4\xbc\x89)]X\x97\xd4q\xbey\x9e\x8d\xd3\xa1p\x17B\x1e\xf4K/\x0f\x01\xec\xbe8F\xe8\xee\x1c\xb4\x98H\x03\xcf\x80\xbf\x93\x91\xf4\xe9\xaeN\xf15\xe8{\xa6\xcd4\\[\xf0\x06\xa0\xbb\xcb\x9f\x8f\x88\xf68\n\x1ftSi\x15\xa5\xc7\xf9\xad\xe2\xe9\x87\xe2e\'\x9d\xce(\xab\x88\xdc6\xa8\t!\xbd\xea\x82\xf8\xa6\xbf\xf32R\xbe\xc21~\xbb\x19s \x8d\x81c\xa2\xe7\xe2\xc7\'\xba\xddaI\x96\x97\xa0G\xe2\x84\x1d\x91\x02\xadw\x15\xa5\xf6\xcd\'\x91\xaf\xcc~5\x10H\'\xbc\xfc4u\xb0\x03\xd7.\x01\xb0\x91_Q-\x80\x10x\xa1\xa1\xfd\xf5\xf5\xc5\xc0#\xc1\x13\x13X\xc0\xbd\x95\xbfam\xea=\x85\x9c\xfa:\xe9\xfa4*\xa9\xe0\xc8,\xd8\xf62^\xb3\xd6\xf9\xfe\xf6\x1b[A\xea(\xa7\xa5\xf1l{)`!\xb4\x1b\xfc\xben\xe2\x01\x16\xbb\x95\xa2GVd\x0f\xc2\x84A$\xa1\x0e\x84-\xd7\xa83\x05\xff\xa2\x04!\x0c\x1bK\xad\xb2\x11(\xa7\xcf\xd5m\xcb>\x9aYU\xa6\xf59\xb9aI\xa1\x066\x0b\xb3!\xd6`\x87n\xb9\xa37\xfb\x9c?\xd8;H\xa9\xac\xc5t\xd2\xde\xf9-\xab\xa9\x14\xdf3\xd2\xa8\x11\xe8\xde\x89>\x82\xe1\x81\x1a\xa0\t\xad\xa7\x0bn\xab\xad\xdd\xb4\x88\'\x9c\x103\xf3\xa9\x17\xf0\xb2\xbf\x04\xc4\xbf\x98\xd6/_&\xc9\xf7\xbc\x89dVa!\xca\xde\x08^p\xcfl<%@\xf5o\x86V\x15\x8c\x9aH\xc4\x98\x0fMk\xad<>\x13V\\\x9b\xd2\xe7A+\xf7\'v\xad\xbe\x9e\xa9\xe5\xe2\xec\xa8\x85\xcf\x04\xae\x94?!#\x9c*\x19\x1d\xc5k\xcb\xcf\xe1*\x83\'\xa8\x93\x8b\x1e\xd7}\x90\x16\xb8a\n\xed\x0b\xb7I\x06\xdf\x8bj\xe8\xd9\xd8\xac\xc4w9\\\xeb\x1f\x9d;2\xdc\xf5^\x84\xc1\n\x1f\'\x11\xfdo\x82\xa7G\\\nv\xd4\x08\x8f\x02\x93hq\xeb\xc3\xbe\xaf\xe9K\xab\xb8`@>2\xd3y\x16"9\xff\xc9\x05\xf9\x12\x8b\xc2\x032\xc9\xf8D\xf5d\xa5\x06\xe0i\xfaZ\xcbQ\xc6Y\xb6G`\x0ee\xad\x93\xc4\x8fhW\xa2x\xe1b\x85\x82Z9\x11\xbff\x92\x1e\xca\xb9K\xb3\xe2$\x01\xe7\x08"\x86\xe925Nx\xb0,\xea\x08R\x92\x08xL\xa2\x8a\xd3\x8d\x98$b\x92\x807bJ\x14\xd5\xf1@\xf3\xee\xc0\x928\x05\xb8\x16V\x89P\xfa\x17\x96_%\xac04\xb2\xfa6\xcdP\ty]\x94\x95\xd4\xcb&m\x00\xfbf\xd7n\xca4x3g\x89\x06\x1ej;\x10$\x93vYt\x1eS\xf8cH\xc0\x15-\xd3\xd7\x8a\xee(\xd8r\x0eF-4\x93\xcd\xf1UOb^\x0f}\x15;\xcd\xbd1\x17H\x02m\xb3B\xb4\x1bOy?N\x93@&\x9a\xdf\xf3cBcr\r\xaee\xdd;\xc6k\xe3J4c-/0\x86,\xa9\x8cG\\\x03f\xc1\x8a_\x0bfS\xe7v\xbd\xf3\x10\xe0I\x8c\xc1\xc1\'\xa9\t\'e\xd8\xf3\x1d\xd5V\xce\xf66)6\xbc=\xd8\xc2oY2\xf4%\xb0\x97{~\x02@\xf5\x04&8V\xd3\xfds\xf6b\xc0\xa4\xc1\x12\xb5>\xb2\xdbY\xf5\x83\x84\x9bm2Y\x81D\x9e\xd0\xcc M\xdbq\xafA(\x92GG\xf6\x17\xc4\xa6\x0e\xa4\r\x83O9\x15\xb1\xf7"\xa5G\xdd\xbbx%4\xd9\xd2E\x18\xc9\x16\xc6Ax\x9f\x14\xcc\xd0\xb0\x81\x90\x96u\xae\x07\x9f\'")\x8b]\xd9\xfa*\xbd\xd4\xach3\x88\xf0\x0fk\xfeh\xd4\xa4,\xdd\x1f\x96\xb9!\x7f/\xa5\x10\x9f\x9d\xb0Z\xec\xbe\x91\xbc\x88#\x04\xd4\x08\x8eE\xe0\x90\xfc\xa9\xbdu\x84\xbaT\xb8f\x19\xc6|\xb7:\x9e\x96gg\xe2\x18/Q\x198\x1a\xa3\xd2\x89\xf7\x19\x19b\xa9$\x18\xfd\x8aa8\xdc\xab\xab@\xc0t\xc3:QI\x1a\\O\xd554\xcda\xd2~\xf1\x0bbI\xa3\x0e\xa5"2\xce\x8d\xee\x9b\x97\xf0\x92\xf5Z^\x12\xe4\xde\x10V\xdf\x9cYQ\xcbP\xe4\xf3*\xc2\xb4^%zV\x03Z,\xe3\xf4\x9e\x1c<\xab\xb2w(\xc0.\x15l\xac\xb5k\xcd\x9a\xf1*\x14\x04M\xdeoq)0;\xebV\xaa=\xb78c\x80\xc4|!|\xc6\xf9\xf2T\x04\x18\xa9f\xf5\xfc\x12\x04\xf4\xce\xaaA\xab*9):`E\xc8\xe0\xa4\xe7\xe1W7\xbcfb\xbbo\x7f\xa9\xeb\xb8\xd5\xa6\xff{(\xdc\x1d\xfdL[\'\t\xa6\x17\x0f\xd4\x80\x02\x9c}0\x0b\x9ai\x04\x19\xbf\x0c\xce\x85\xfc\xab\x1am\xe5\xc6`<\x1d\x8d\xfc\xd6\x1f\xa0\x84\xa1\xc7\xe3\x06m\x98~\xd6I0\xa7\xf7(Z@"\x1f\xfe@\xb0@\xdc\t\x16D\xffTT\xfa\x8a\'\x01N\xb8\xaf\xd1\xd5\xff\x1cr\xec\xc9\r\xba\xef#\xd9\xec\xd8u_\xce\xfcC \x7f0w\xca>\xfa"\xd4IR\xfd\x9c$B\xf7\x00_d\xf4\xe2$\xbdP\x94\x1f\\lQ\xb5\xd0 \x84\xadt\x95y\x18\xa9L~\xc0>\xb5\xab\x9fbM\x92FC\'U\x90\x05E\xeb\x90\xf2\xc8\x8bw\x9b\xb6\xe3\x9e\xd5R\x87\x02?]\xba\xab.\x0b\x97,W\x96\x8b>\xb4\xcb[1P\xb7K\xb7\xbc\xde\xec\x9b*\xed\xcd\xc8\x9d\xd6@\x86u\xfe\x0c\xa0\x11\x1b\xb3\x95\xba\xe9\x93\xeb\xc4BKQ\xff\xf4\xd9+|\xc6\x97v\x7f\xd5\x1b\x0e\xb15\x931(j\xab\x80\x1a\xcb6\x834\x86;\xc7\x16@\xe2%\xdfhu\xce\x8f\xf2e\xd02\xa1\nU\xa0\n0\x9f\xd5(W\xcar\x08\xfc+M[^\xcd\x9b\x8f\x14\x8a\xbd\x9c\xe3\x01\x84\x00\xac\x19\xfd\x1b\xf1\xdd\xa6\x8a\x9aG\xecj\xe1(#\x83\x8b\xa3\xdb\xa88\xfe\x85w\xb3@)\xc0\xa0\'\x82\x15aeX5V\x86P\xabU\xb0\x8eIp\xfb\x88\\N\xe5\x1dl\x9f\xf8\xe7"\xda\xca\x85\x8c(i\x04c(\xcf\x91bb\xc6\x1f\xe9\x1c\xf1\xday\xf9\xef99$\xae\xe6\x84y\xe1N\x17o\x1bRp\x84<#G_\xb0M:DY,H\xa8/\x12\x9e\xf6\x037\x08\xe0t\xaa\r\x1359\x199\x94\x13n|K\xe5\xf44\xd0\xc9seb\x1ft\xd1\x05 >\x9a\xbcC3,\xda\xdc\xce\xdc\x8e\xb8\xc8Dx\x18\xae\x01\x05\xb4\x96d?\xc7n\xf6\x17\x04\xb2\xa3\xee\x86\xca\x86\xdb\xedh\xfd9\x12\xed\xa9\xe3\xe8\x85\x036\x142/\xd0\x98fZ\xddqg\xfe\xed+o\xfe\xfd\xd9\xd2\xf9]\xafG\xf4\'\x9a\xfa\x10\xd9\xf5\xceXE\xa2?\x11\xc5\xf8\xdc%\xdf\x84\x18\x88\xf9\xab\x12\xc2,\xa7\xe1\xa7\xce\xa0\xfe\x99^D\xabi\xaf\xbf\x0fQ\x95\x02\tz"\x83kd\xd4M\xb2]\x93\xb19\x90kf2\x93\x17\xb0\x8a\xba\xbf\xa1\xc4}[\x91\xb8\xda8\x1b\x81\x0f\xef\x8b\xaaa\xfb\xdb],1IX\xc1[(\xca3\x837\xd9\xb6\x97Z\x97\xe7#i\x92\x1f\x88\x06\x07eT\x1d$C;\xb8h\xb1\x92\xce]\'7\xb0\xd4\x87\xbf\xb0\xe1\xaao\x18\xe5_\r\x8a\xd9\xa5"\xa8\xd9\x07\xa9\x9a\x8c\xf7=\x9aK\x0e1\n\xb8\xd7\xaf \n\\~\'\xdb\\`\xd3\x1b\x07\xb2\xea@\xc1\xba&\xa7\xbd\\H?%\x1e\x83(\x12*\xd5\xc0\x1a\xe3w\xc3Ik\xb3\xd2M~\xa1V\x15\x01H&\xbf;b\x16\xb5]\xc1\xf6\xc9\xdc\xe7\x1e=\x0b!&&\xb6_8M\x9b\xa93j\x83b\x9c\x06\x1f\xb0\xdbu);o\xd7}\x0f\xb9T\x89`\xe8\xec\xcb\x8c!6\xff\xda\\\x87\xd5Uns\x08\xa8)aqIo\xf1\xb8\x1b2ZZ\'N\x9cr\x0fbq\xa3\xb56LZ\x839F\xe3\xc8\xd2I\xdf\xd3-\xd7n\xf1\xd6\x16%\xf9k\x8fQ\x90\xc8Qd*\x0c\x008\x96^;\x9d\xd1\xce\xc5\xee\xd9\xb5R\xec\x1452R5\x8a\x12\x82Y<K\x80z\x88\x9e&\x8b\xf4l^\x99\x1f:\xa5\xc52\xda\xad\xea}\xd1\x95g9\xb0\xc1\x1f\xccL\x94 CQ\xcaVm|\xbdd\xb1\xaf\x1aY\xb6(\x07\xf9\x94\x96\xaa\x86X^ji\x13\xc5\x04\xc1\x18\xbc\x8c\x83\xe6\xf5 \xe9^h\xf7\xbf\xc5\x80\xa2\x1bI\xd2\xec+\xa5\xaa\xfc0\x8fsy\x15\xdc\xc7f\x81\xf2\x99\x86\x14\xb8\xe2\xed\xb7\xf5&\\\xac~t\x1c\x04b\xd28?\x1d@e\xbd\x10\x18\x1b-;ARaen\x83\xa3\xa9\xaf\xc3\xdc\x00\t\n\x10\x12\x16"\'>Zemvxz\x80\xe9#knrz\xb8\xbb\xd2\xf8\x0b\x160Jalrt\x85\x8b\xa9\xb2\xbf\xe0\xe9\xf1\xf2\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x10!*;'

In [13]:
trig_count = the_offset

In [14]:
glitch_spots = [i for i in range(1)]
# ###################
# Add your code here
# ###################
#raise NotImplementedError("Add your code here, and delete this.")

# ###################
# START SOLUTION
# ###################
glitch_spots = list(range(trig_count - 50, trig_count + 50, 1))
if PLATFORM == "CWLITEXMEGA":
    glitch_spots = list(range(9500, 9650, 1))
# ###################
# END SOLUTION
# ###################

In [15]:
print(glitch_spots)

[948, 949, 950, 951, 952, 953, 954, 955, 956, 957, 958, 959, 960, 961, 962, 963, 964, 965, 966, 967, 968, 969, 970, 971, 972, 973, 974, 975, 976, 977, 978, 979, 980, 981, 982, 983, 984, 985, 986, 987, 988, 989, 990, 991, 992, 993, 994, 995, 996, 997, 998, 999, 1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023, 1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1032, 1033, 1034, 1035, 1036, 1037, 1038, 1039, 1040, 1041, 1042, 1043, 1044, 1045, 1046, 1047]


In [16]:
scope.adc.timeout = 0.1
        
scope.glitch.clk_src = 'clkgen'
scope.glitch.trigger_src = 'ext_single'
scope.glitch.repeat = 1
scope.glitch.output = "clock_xor"
scope.io.hs2 = "glitch"

In [17]:
import matplotlib.pylab as plt
import chipwhisperer.common.results.glitch as glitch
gc = glitch.GlitchController(groups=["success", "reset", "normal"], parameters=["width", "offset"])
gc.display_stats()


#fig = plt.figure()
#plt.plot(-48, 48, ' ')
#plt.plot(48, -48, ' ')
#plt.plot(-48, -48, ' ')
#plt.plot(48, 48, ' ')

DEBUG:matplotlib:matplotlib data path: /home/alexis/.venvs/chipwhisperer.dilithium/lib/python3.8/site-packages/matplotlib/mpl-data
DEBUG:matplotlib:CONFIGDIR=/home/alexis/.config/matplotlib
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is linux
DEBUG:matplotlib:CACHEDIR=/home/alexis/.cache/matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from /home/alexis/.cache/matplotlib/fontlist-v330.json


IntText(value=0, description='success count:', disabled=True)

IntText(value=0, description='reset count:', disabled=True)

IntText(value=0, description='normal count:', disabled=True)

FloatSlider(value=0.0, continuous_update=False, description='width setting:', disabled=True, max=10.0, readout…

FloatSlider(value=0.0, continuous_update=False, description='offset setting:', disabled=True, max=10.0, readou…

In [18]:
faulted_sigs = []

In [19]:
from importlib import reload
import chipwhisperer.common.results.glitch as glitch
from chipwhisperer.capture.targets import TargetTimeoutError, TargetIOError
from tqdm.notebook import tqdm
import re
import struct
import numpy as np
# ###################
# These width/offset settings are for CW-Lite/Pro; width/offset are expressed differently for Husky (see Fault 1_1)
gc.set_range("width", 2, 14)
gc.set_range("offset", -14, 14)

if PLATFORM == "CWLITEXMEGA":
    gc.set_range("width", 46, 49.8)
    gc.set_range("offset", -46, -49.8)
if PLATFORM == "CW308_STM32F4":
    #gc.set_range("width", 0.4, 10)
    #gc.set_range("offset", 40, 49.8)
    gc.set_range("width", 3, 9)
    gc.set_range("offset", 0, 8)
step = 1
gc.set_global_step(step)
#scope.glitch.repeat = 4
scope.glitch.repeat = 1
target.reboot_flush()

broken = False
for glitch_setting in gc.glitch_values():
    scope.glitch.offset = glitch_setting[1]
    scope.glitch.width = glitch_setting[0]
    for i in tqdm(glitch_spots, leave=False):
        scope.glitch.ext_offset = i
        #if broken:
        #    break
        #if scope.adc.state:
            #print("Timeout, trigger still high!")
        #    gc.add("reset", (scope.glitch.width, scope.glitch.offset))
            #plt.plot(scope.glitch.width, scope.glitch.ext_offset, 'xr', alpha=1)
            #fig.canvas.draw()

            #Device is slow to boot?
       #     target.reboot_flush()
        #target.flush()
        scope.arm()
        try:
            target.sign(the_msg, timeout=700)
        except TargetTimeoutError as e:
            gc.add("reset", (scope.glitch.width, scope.glitch.offset))
            print('reset due timeout')
            target.reboot_flush()
            continue
        except TargetIOError as e:
            gc.add("reset", (scope.glitch.width, scope.glitch.offset))
            print(f'reset with TargetIOError error: {e}')
            target.reboot_flush()
            
        ret = scope.capture()
        if ret:
            print('Timeout - no trigger')
            gc.add("reset", (scope.glitch.width, scope.glitch.offset))
            #plt.plot(scope.glitch.width, scope.glitch.ext_offset, 'xr', alpha=1)
            #fig.canvas.draw()

            #Device is slow to boot?
            target.reboot_flush()
        else:
            #time.sleep(0.05)
            candidate_sig = target.get_sig()
            _, z_candidate, _ = dilithium.generic._unpack_sig(candidate_sig, nist_security_level=2)
            if 0 < np.sum(z_candidate == z_correct) < 1024: # not all differen but also not all same
                faulted_sigs += [candidate_sig]
                print("Glitched!\n\tExt offset: {}\n\tOffset: {}\n\tWidth: {}".format(i, scope.glitch.offset, scope.glitch.width))
                #plt.plot(scope.glitch.width, scope.glitch.ext_offset, '+g')
                gc.add("success", (scope.glitch.width, scope.glitch.offset))
                #fig.canvas.draw()
                #broken = True 
                #for __ in range(500):
                #    num_char = target.in_waiting()
                #    if num_char:
                #        my_print(output)
                #        output = target.read(timeout=50)
                #time.sleep(1)
                break
            else:
                gc.add("normal", (scope.glitch.width, scope.glitch.offset))

(ChipWhisperer Glitch WARNING|File ChipWhispererGlitch.py:770) Partial reconfiguration for offset = 0 may not work
(ChipWhisperer Glitch WARNING|File ChipWhispererGlitch.py:770) Partial reconfiguration for offset = 0 may not work


  0%|          | 0/100 [00:00<?, ?it/s]

DEBUG:root:Trigger found at 91
DEBUG:root:Trigger found at 85
DEBUG:root:Trigger found at 91
DEBUG:root:Trigger found at 106
DEBUG:root:Trigger found at 80
DEBUG:root:Trigger found at 105
DEBUG:root:Trigger found at 95
DEBUG:root:Trigger found at 85
DEBUG:root:Trigger found at 91
DEBUG:root:Trigger found at 90
DEBUG:root:Trigger found at 102
DEBUG:root:Trigger found at 76
DEBUG:root:Trigger found at 85
DEBUG:root:Trigger found at 102
DEBUG:root:Trigger found at 84
DEBUG:root:Trigger found at 82
DEBUG:root:Trigger found at 109
DEBUG:root:Trigger found at 86
DEBUG:root:Trigger found at 94
DEBUG:root:Trigger found at 101
DEBUG:root:Trigger found at 88
DEBUG:root:Trigger found at 92
DEBUG:root:Trigger found at 79
DEBUG:root:Trigger found at 108
DEBUG:root:Trigger found at 89
DEBUG:root:Trigger found at 91
DEBUG:root:Trigger found at 76
DEBUG:root:Trigger found at 102
DEBUG:root:Trigger found at 95
DEBUG:root:Trigger found at 109
DEBUG:root:Trigger found at 97
DEBUG:root:Trigger found at 99

  0%|          | 0/100 [00:00<?, ?it/s]

DEBUG:root:Trigger found at 86
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 


...

Glitched!
	Ext offset: 958
	Offset: 3.125
	Width: 7.03125


  0%|          | 0/100 [00:00<?, ?it/s]

DEBUG:root:Trigger found at 105
(ChipWhisperer Target WARNING|File SimpleSerial2.py:502) Read timed out: 


reset due timeout


DEBUG:root:Trigger found at 104
DEBUG:root:Trigger found at 77
DEBUG:root:Trigger found at 82
IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [20]:
print("Glitched!\n\tExt offset: {}\n\tOffset: {}\n\tWidth: {}".format(i, scope.glitch.offset, scope.glitch.width))
print(candidate_sig)

Glitched!
	Ext offset: 1047
	Offset: 7.8125
	Width: 8.984375
b'l\x15\x97\xe4\xe8<w\xca\x87c\xc1g^%o\xb16\x0b\x8e\xc3\xef\xf9\x10\xd8\xe4\xd5\xbe\x1b,T\xcb\xb3\xe1\x87K\x07\x84\xa8\x1d\x83G\xaat\x01\x8f\xd1\x0b\xe3\xc3\xad\x0c\x91\x92m;%\xa5\xb7}\xa8<|F\xf5t\x81\x7f|\x95\xeel\xfc,\xa9\xef?\n\xf9\xf6e\x08\xd0\x02_\xac\xa87\xb5\x94\x83h\xbbM\xf3\x14\x11e\xbb\x96-\xb4\x83\xc2hIiy\x0e\xd2\xae\xb0\x9frHr\xde\xb8\xbd\xc6Y6d\xb5\x82\xb3)*\x8a?\x1f\xa3\xdb\x81c\xaeGL\xc2I\xa3\xd7W5F\xf0y\xac1\x9a\xc3\xa7@\xfb\xaa\x95!\x7f&D\x97\x05\x7f\x00\x84\x19^<v\x9f\xc9\xa1\x9b\x93\x08\xceX\xec8\x86\xec\x14\x96;\x1f\xc3\xa5\xfcP\xc6\x1c"u\x10\xcbL\x03\xf9\xe8\xd9^\xb0\xbb\xf5\xc5o7\xfc\x18\x13*\xf6\xe8\x83\x1d\x18T\xf2>+v\xe8\xc4\x1c\xf8L\xfb\x9c\xf0\x1e\x9b\xc8\x8a\x04\x18\xea]\xca\\\x90\x0b\xa0\x13;\x01`sG\xed-\xf2\x12Q:\r\xf80\xb6\xecf\xc9\x7f\xca\xf5\'\x845\xe37\xd1\x8eZ\x06\xd0\xd4\x8f1&iA\xa2\xbcS\xb9\n\xb0\xf6\xd7\xc2]\x02o\x1e\x93\x82[K\xb0|P\x9f\xec&\x11\xdcB@/\xa4\xf7Qq\xe3\x0bl^\xe4K\xe4B\xa3\x0

In [40]:
for the_faulted in faulted_sigs:
    _, unpacked_faulted_z, _ = dilithium.generic._unpack_sig(the_faulted, nist_security_level=2)
    sum_0 = np.sum(unpacked_faulted_z[0] == z_correct[0])
    sum_last = np.sum(unpacked_faulted_z[-1] == z_correct[-1])
    print(sum_0)
    print(sum_last)
    print()
    if sum_0 > 0 and sum_last == 256:
        for i in range(len(unpacked_faulted_z)):
            print(unpacked_faulted_z[i] == z_correct[i])


7
4

7
4

7
4

7
4

7
4

7
4

7
4

7
4

7
4

7
4

7
4

7
4

7
4

7
4

7
4

7
4

7
4

7
4

7
4

7
4

7
4

7
4

7
4

9
5

9
5

7
4

7
4

7
4

7
4

7
4

7
4

9
5

5
5

7
4

7
4

7
4

7
4

7
4

4
4

7
4

7
4

7
4

7
4

8
11

5
5

7
4

7
4

7
4

8
4

8
7

5
5



In [22]:
import numpy as np
print(len(faulted_sigs))
f_sig = faulted_sigs[-1]
_, z_faulted, _ = dilithium.generic._unpack_sig(f_sig, nist_security_level=2)
_, z_correct, _ = dilithium.generic._unpack_sig(correct_sig, nist_security_level=2)
np.sum(z_correct == z_correct)

51


1024

In [32]:
print(gc.__dict__['results'].__dict__['result_dict'])

{'success': [{'parameters': (3.125, 1.171875), 'strdesc': None, 'metadata': None}, {'parameters': (3.125, 1.953125), 'strdesc': None, 'metadata': None}, {'parameters': (3.125, 3.90625), 'strdesc': None, 'metadata': None}, {'parameters': (3.125, 5.078125), 'strdesc': None, 'metadata': None}, {'parameters': (3.125, 5.859375), 'strdesc': None, 'metadata': None}, {'parameters': (3.125, 7.03125), 'strdesc': None, 'metadata': None}, {'parameters': (3.125, 7.8125), 'strdesc': None, 'metadata': None}, {'parameters': (3.90625, 0.0), 'strdesc': None, 'metadata': None}, {'parameters': (3.90625, 1.171875), 'strdesc': None, 'metadata': None}, {'parameters': (3.90625, 1.953125), 'strdesc': None, 'metadata': None}, {'parameters': (3.90625, 3.125), 'strdesc': None, 'metadata': None}, {'parameters': (3.90625, 3.90625), 'strdesc': None, 'metadata': None}, {'parameters': (3.90625, 5.078125), 'strdesc': None, 'metadata': None}, {'parameters': (3.90625, 5.859375), 'strdesc': None, 'metadata': None}, {'para

## Diagnosing the Fault

As you can see by the output, the bootloader has suffered a pretty catastrophic failure! Not only has it spilled the secret, it's also dumped a whole bunch more memory. For a real bootloader, there's probably some pretty juicy stuff in there like encryption keys or previously decrypted firmware. Let's start by taking a look at the C source code that sends the error code back:

```C
trigger_high();

int i;
for(i = 0; i < ascii_idx; i++)
{
    putch(ascii_buffer[i]);
}
trigger_low();
state = IDLE;
```

Nothing really looks too unusual here. Before we take a look at the assembly and figure out what went wrong, let's try to make some guesses:

* Maybe the glitch corrupted the `ascii_idx` variable
    * The glitch happened near the end of the loop. It's unlikely the end of loop counter would be reloaded during the loop
* Maybe we skipped the last `i < ascii_idx` check
    * The glitch caused **a lot** of memory to be dumped. If we just skipped the last check it **should** only print an extra character
* i is a signed integer: maybe we corrupted it into being a really large negative number.

That last one seems to be our best theory, so let's go with that.

## The Answer

Let's check the assembly for our booloader. No need to decompile the binary or recompile to assembly, since there's also a listing file created as part of the build process (`*.lss`). This file also contains C, so it makes it easy to search (try something like the `trigger_high()` call). You might notice that instead of doing a `less than or equal` or `less than` comparison like was in our C code, the compiler has instead inserted a `not equal` comparison instead! This means our original guess may not have been correct, as our assumption about what would happen if the last `i < ascii_idx` was skipped doesn't hold. In fact, it's a lot more likely that the last check was skipped (or i was set to some large value) than flipping a particular bit.

This is actually a pretty unexpected change for the compiler to make, espcially since `less than`, `greater than`, and `not equal` are nearly identical instructions in terms of implementation and have both the same instruction size and speed. This showcases an important fact: the C code that you write is not directly translated to assembly. It needs to go through the compiler first, which may drastically change the intended logic of the program.

Now that we know what happened, let's look at some ways to fix it.

### 1. Volatile variables

C includes a keyword for variables called `volatile`, which indicates that the variable may change between accesses and therefore should not have optimizations applied to it. A typical use case for `volatile` is for peripheral registers on embedded devices. It would be really bad, for example, if you were trying to wait for an IO pin to go high in your code, but the compiler decided it would be faster to only check it only once and assume it doesn't change!

Try replacing `int i = 0;` before the print look with `volatile int i = 0;`, recompile, and check the listing file. Is there any other unexpected changes? What about if you consider the use case above (i.e. if `i` was a register instead of a loop variable)? Is there any way the attack might still work? If so, how might you mitigate this?

### 2. Unrolling the loop

Another potential way of solving this issue would be to manually unroll the loop. The message being printed by the bootloader is a constant length of 7 characters, so we could instead write:

```C
int i;
putch(ascii_buffer[i++]);
putch(ascii_buffer[i++]);
putch(ascii_buffer[i++]);
putch(ascii_buffer[i++]);
putch(ascii_buffer[i++]);
putch(ascii_buffer[i++]);
putch(ascii_buffer[i++]);
```

In fact, this is something the compiler might do on its own to optimize the code, since unrolling a loop like this is faster than the loop version. It's not a good idea to blindly rely on this, however, since the compiler could choose not to make this optimization as well and might change it between builds.

### 3. Checking for invalid characters

Another thing to consider is that the message from the bootloader only has a limited range of characters that it prints. We could instead construct a "safe print" function that only prints newlines, `'r'` and ASCII digits (i.e. `'0'` to `'9'`):

```C
int safe_print(char c)
{
    if ((c == '\n') ||
       ((c >= '0') && (c <= '9')) ||
       (c == 'r')) {
        putch(c);
        return 0;
    }
    return -1; //uh oh!
}
```

It we went this route, it would be a good idea to make the error return a separate buffer with a bunch of null characters at the end.

### 4. More generic methods

More generic ways of defending against glitch attacks (memory guards, for example) are also discussed in the training slides.

In [41]:
scope.dis()
target.dis()

(ChipWhisperer Scope ERROR|File naeusbchip.py:108) Scope already disconnected!


In [24]:
assert broken is True

AssertionError: 